## Env

In [137]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.init
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from tqdm import tqdm

In [138]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## Data Load

In [139]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, image_folder, transform=None):
        self.df = pd.read_csv(csv_file)
        self.image_folder = image_folder
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = self.df['img_path'].iloc[idx].split("/")[-1]
        img_path = os.path.join(self.image_folder, img_name)
        image = Image.open(img_path)

        if self.transform:
            image = self.transform(image)

        return image

In [146]:
CFG = {
    'EPOCHS':300,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':16,
    'SEED': 42
}

In [147]:
def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED'])

In [148]:
train_transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.RandomHorizontalFlip(p = 0.5),
    # transforms.RandomVerticalFlip(p = 0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_data = CustomDataset(csv_file='/content/drive/MyDrive/Project/반도체 소자/train.csv',
                           image_folder='/content/drive/MyDrive/Project/반도체 소자/train',
                           transform = train_transform)
test_data = CustomDataset(csv_file='/content/drive/MyDrive/Project/반도체 소자/test.csv',
                           image_folder='/content/drive/MyDrive/Project/반도체 소자/test',
                           transform = test_transform)

train_loader = DataLoader(train_data, batch_size = CFG['BATCH_SIZE'], shuffle = True)
test_loader = DataLoader(test_data, batch_size = CFG['BATCH_SIZE'], shuffle = False)

In [149]:
class ConvAutoencoder(nn.Module):
  def __init__(self):
    super(ConvAutoencoder, self).__init__()

    self.encoder = nn.Sequential(
        nn.Conv2d(3, 32, kernel_size = 7, stride = 2, padding = 3),
        nn.Mish(),
        nn.BatchNorm2d(32),
        nn.Conv2d(32, 32, kernel_size = 1),
        nn.Mish(),
        nn.BatchNorm2d(32),
        nn.Conv2d(32, 64, kernel_size = 3, stride = 2, padding = 1),
        nn.Mish(),
        nn.BatchNorm2d(64),
        nn.Conv2d(64, 64, kernel_size = 1),
        nn.Mish(),
        nn.BatchNorm2d(64),
        nn.Conv2d(64, 128, kernel_size = 3, stride = 2, padding = 1),
        nn.Mish(),
        nn.BatchNorm2d(128),
        nn.Conv2d(128, 128, kernel_size = 1),
        nn.Mish(),
        nn.BatchNorm2d(128),
        nn.Conv2d(128, 256, kernel_size = 3, stride = 2, padding = 1),
        nn.Mish(),
        nn.BatchNorm2d(256),
    )
    self.decoder = nn.Sequential(
        nn.ConvTranspose2d(256, 128, kernel_size = 3, stride = 2, padding = 1, output_padding = 1),
        nn.Mish(),
        nn.BatchNorm2d(128),
        nn.ConvTranspose2d(128, 128, kernel_size = 1),
        nn.Mish(),
        nn.BatchNorm2d(128),
        nn.ConvTranspose2d(128, 64, kernel_size = 3, stride = 2, padding = 1, output_padding = 1),
        nn.Mish(),
        nn.BatchNorm2d(64),
        nn.ConvTranspose2d(64, 64, kernel_size = 1),
        nn.Mish(),
        nn.BatchNorm2d(64),
        nn.ConvTranspose2d(64, 32, kernel_size = 3, stride = 2, padding = 1, output_padding = 1),
        nn.Mish(),
        nn.BatchNorm2d(32),
        nn.ConvTranspose2d(32, 32, kernel_size = 1),
        nn.Mish(),
        nn.BatchNorm2d(32),
        nn.ConvTranspose2d(32, 3, kernel_size = 3, stride = 2, padding = 1, output_padding = 1),
        nn.Sigmoid(),
    )

  def forward(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)

    return decoded

In [150]:
def train_autoencoder(model, dataloader, criterion, optimizer, num_epochs):
    model.to(device)

    for epoch in range(num_epochs):
        running_loss = 0.0

        for images in dataloader:
            images = images.to(device)
            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, images)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)

        epoch_loss = running_loss / len(dataloader.dataset)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

    print("Training complete!")

In [151]:
model = ConvAutoencoder()

criterion = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr = CFG['LEARNING_RATE'])

train_autoencoder(model, train_loader, criterion, optimizer, num_epochs = CFG['EPOCHS'])

Epoch 1/300, Loss: 1.9061
Epoch 2/300, Loss: 1.7977
Epoch 3/300, Loss: 1.7259
Epoch 4/300, Loss: 1.6769
Epoch 5/300, Loss: 1.6385
Epoch 6/300, Loss: 1.6057
Epoch 7/300, Loss: 1.5754
Epoch 8/300, Loss: 1.5483
Epoch 9/300, Loss: 1.5242
Epoch 10/300, Loss: 1.5026
Epoch 11/300, Loss: 1.4832
Epoch 12/300, Loss: 1.4656
Epoch 13/300, Loss: 1.4493
Epoch 14/300, Loss: 1.4342
Epoch 15/300, Loss: 1.4198
Epoch 16/300, Loss: 1.4072
Epoch 17/300, Loss: 1.3948
Epoch 18/300, Loss: 1.3827
Epoch 19/300, Loss: 1.3718
Epoch 20/300, Loss: 1.3615
Epoch 21/300, Loss: 1.3515
Epoch 22/300, Loss: 1.3429
Epoch 23/300, Loss: 1.3338
Epoch 24/300, Loss: 1.3250
Epoch 25/300, Loss: 1.3165
Epoch 26/300, Loss: 1.3084
Epoch 27/300, Loss: 1.3005
Epoch 28/300, Loss: 1.2937
Epoch 29/300, Loss: 1.2865
Epoch 30/300, Loss: 1.2798
Epoch 31/300, Loss: 1.2721
Epoch 32/300, Loss: 1.2658
Epoch 33/300, Loss: 1.2589
Epoch 34/300, Loss: 1.2529
Epoch 35/300, Loss: 1.2470
Epoch 36/300, Loss: 1.2408
Epoch 37/300, Loss: 1.2352
Epoch 38/3

In [152]:
submit = pd.read_csv('/content/drive/MyDrive/Project/반도체 소자/sample_submission.csv')
submit.head()

,id,label
0,TEST_000,0
1,TEST_001,0
2,TEST_002,0
3,TEST_003,0
4,TEST_004,0


In [158]:
model.eval()

loss_ = []

with torch.no_grad():
    for images in test_loader:
        images = images.to(device)
        outputs = model(images)
        loss = torch.mean(torch.square(outputs - images), dim = (1, 2, 3))
        loss_.extend(loss.cpu().numpy())
    loss_ = np.array(loss_)

In [160]:
print(loss_)


[0.9766984  0.92330384 0.96261275 0.87479675 0.9726679  0.9240381
 0.8926565  0.9690161  0.939058   0.85723853 0.8804711  0.93933487
 0.88733435 0.8733846  0.91061866 0.98398864 0.9875827  0.8875303
 0.8648092  0.9995121  0.91529644 0.94359607 0.8923251  0.9686212
 0.91607785 0.99617815 0.87421703 0.95833725 0.96590513 1.0073266
 0.85408056 0.95696867 0.8948769  0.91779375 0.476966   0.9982834
 0.92447203 0.9246186  0.97617674 0.93161285 1.0118905  0.97754216
 1.0212193  0.89713997 0.9602229  1.045061   0.9354062  0.9283551
 0.9416167  0.9001733  1.0348753  0.93343973 0.96961844 0.92375225
 0.9908396  0.9154322  0.9565281  0.9499434  0.9516306  0.9157385
 0.9505634  0.9718708  0.8950225  0.87747353 0.8990793  0.9410154
 0.9311225  0.9774997  0.9907901  0.9054152  1.0463675  0.9733154
 0.9291374  1.0139341  0.9223156  0.9564039  0.9032232  0.87944996
 0.99662733 0.9245311  0.96412706 0.95899653 0.9702209  0.9462588
 0.9856733  0.959249   0.9033581  0.96421885 0.92454183 0.9474567
 0.962

In [162]:
threshold = 0.9

# print(loss_)

loss_ = np.where(loss_ > threshold, 1, 0)

print(loss_)
submit['label'] = loss_

[1 1 1 0 1 1 0 1 1 0 0 1 0 0 1 1 1 0 0 1 1 1 0 1 1 1 0 1 1 1 0 1 0 1 0 1 1
 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 0 1]


In [163]:
submit['label'].value_counts()

1    79
0    21
Name: label, dtype: int64

In [156]:
# submit['label'] = submit['label'].replace({0 : 1. 1 : 0})

# submit['label'].value_counts()

In [157]:
submit.to_csv('/content/drive/MyDrive/Project/반도체 소자/mish_01.csv', index = False)